# Exercise 2-2: Write your own code for the Mortality Notebook

In [788]:
import pandas as pd

### Get data

In [789]:
# my example didn't have a pkl file, so I made it  :)
mortality_url = "https://data.cdc.gov/api/views/v6ab-adf5/rows.csv?accessType=DOWNLOAD"
mortality_data = pd.read_csv(mortality_url)

### Clean and prep the data

In [790]:
# remove spacing in column headers > would need to handle camel case
mortality_data.columns = mortality_data.columns.str.replace(" ", "")

#calculate the mean centered value, normalizing the data
mortality_data['MeanCentered'] = \
    mortality_data.DeathRate - mortality_data.DeathRate.mean()

# make all single digit values two-digit for easy printing
mortality_data.AgeGroup.replace(
    {'1-4 Years':'01-04 Years','5-9 Years':'05-09 Years'},
    inplace = True)

# pivot table to get wide table
mortality_wide = mortality_data.pivot(
    index = "Year", columns="AgeGroup" ,values=["DeathRate", "MeanCentered"])

# reset the index to allow for mortality_long melting
mortality_wide.reset_index(inplace=True)

# set mortality long based on wide dataset
mortality_long = mortality_wide.melt(
    id_vars='Year',
    var_name='Year',
    value_vars=['01-04 Years','05-09 Years','10-14 Years','15-19 Years'],
    value_name="DeathRate"
)

# rename per directions -- is there any way to query this as a class??
mortality_data.columns = mortality_data.columns.str.replace("DeathRate","DeathRate/100K")
mortality_data.tail(8)

C:\Users\graha\AppData\Local\Temp/ipykernel_17168/2249503237.py:21: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  mortality_long = mortality_wide.melt(


KeyError: 'Year'

# Wide DataFrame

### Review dataframe

In [ ]:
# display df head
mortality_wide.head(5)

In [ ]:
# show first 5 years and MeanCentered
mortality_wide.Mean

In [ ]:
# get summary
mortality_wide.describe()

In [ ]:
# transpose summary
mortality_wide.describe().T

In [ ]:
# isolate columns
mortality_wide.loc[:, ['Year', '01-04 Years']]

In [ ]:
# query wide dataframe
mortality_wide.query('Year >= 1915 and  Year <= 1920')

In [ ]:
# query dataframe and isolate columns for display/report
mortality_wide.query('Year >= 1915 and  Year <= 1920').loc[:, ['Year', '01-04 Years']]

In [ ]:
# Get mean, median and sum for each year

# add new column called TotalDeaths: rolling sum of deaths for each year

# create line plot to shot total death rate by year

# Long Dataframe

In [ ]:
# set mean centered
mortality_long["MeanCentered"] = \
    mortality_long.DeathRate - mortality_long.DeathRate.mean()

# set median
death_rate_median = mortality_long.DeathRate.median()

# replace header per spec
# mortality_long.columns = mortality_long.columns.str.replace("DeathRate","DeathRate/100K")

mortality_long

### Review dataframe

In [ ]:
# display top 5 w/ mean centered
mortality_long.head(5)

In [ ]:
# median calculated in long dataframe setup
print(f"Median death rate: {death_rate_median}")

In [ ]:
# last 6 rows of data from 1915 to 1920
mortality_long.query("Year >= 1915 and Year <= 1920").tail(6)

In [ ]:
# year and deaths/100k for 01-04 years
mortality_long.set_index('Year', inplace=True)
mortality_long.query("AgeGroup == '01-04 Years'").loc[:, ['AgeGroup', 'DeathRate']]

In [ ]:
# show dataframe in descending order
mortality_long.sort_values('DeathRate', ascending=False)

# create smaller dataframe to create a subset of the first 3 and last 3
report_df = mortality_long.head(3)
report_df = report_df.append(mortality_long.tail(3))
# display dataframe
report_df

### Grouped sums

In [ ]:
# as-written to "group" by year and sum the rate
mortality_long.groupby('Year').sum(['DeathRate'])

In [ ]:
# how I read the problem
mortality_long.DeathRate.cumsum()